In [ ]:
import tensorflow as tf


In [3]:
from google.colab import drive
drive.mount('/content/drive')  # ✅ correct


Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/Project_Dataset/cropped /content/


In [ ]:
data_dir = "/content/cropped"

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import os

In [ ]:
DATA_DIR = '/content/dataset'   # Your dataset root directory
IMG_SIZE = 224
BATCH_SIZE = 128
EPOCHS_INITIAL = 30
EPOCHS_FINE_TUNE = 10

In [ ]:
 #1. Load dataset with 80-20 train-validation split
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

Found 26237 files belonging to 299 classes.
Using 20990 files for training.


In [ ]:
# Save class names immediately
class_names = train_ds.class_names


In [ ]:
class_names

['n000002',
 'n000003',
 'n000004',
 'n000005',
 'n000006',
 'n000007',
 'n000008',
 'n000010',
 'n000011',
 'n000012',
 'n000013',
 'n000014',
 'n000015',
 'n000016',
 'n000017',
 'n000018',
 'n000019',
 'n000020',
 'n000021',
 'n000022',
 'n000023',
 'n000024',
 'n000025',
 'n000026',
 'n000027',
 'n000028',
 'n000030',
 'n000031',
 'n000032',
 'n000033',
 'n000034',
 'n000035',
 'n000036',
 'n000037',
 'n000038',
 'n000039',
 'n000041',
 'n000042',
 'n000043',
 'n000044',
 'n000045',
 'n000046',
 'n000047',
 'n000048',
 'n000049',
 'n000050',
 'n000051',
 'n000052',
 'n000053',
 'n000054',
 'n000055',
 'n000056',
 'n000057',
 'n000058',
 'n000059',
 'n000060',
 'n000061',
 'n000062',
 'n000063',
 'n000064',
 'n000065',
 'n000066',
 'n000067',
 'n000068',
 'n000069',
 'n000070',
 'n000071',
 'n000072',
 'n000073',
 'n000074',
 'n000075',
 'n000076',
 'n000077',
 'n000079',
 'n000080',
 'n000081',
 'n000083',
 'n000084',
 'n000085',
 'n000086',
 'n000087',
 'n000088',
 'n000089',
 'n0

In [ ]:
def preprocess(image, label):
    image = preprocess_input(image)
    return image, label


In [ ]:
# ✅ Then apply preprocessing and prefetch
train_ds = train_ds.map(preprocess).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Now you can use class_names anywhere in your code
print("Classes:", class_names)
num_classes = len(class_names)
print(num_classes)

Classes: ['n000002', 'n000003', 'n000004', 'n000005', 'n000006', 'n000007', 'n000008', 'n000010', 'n000011', 'n000012', 'n000013', 'n000014', 'n000015', 'n000016', 'n000017', 'n000018', 'n000019', 'n000020', 'n000021', 'n000022', 'n000023', 'n000024', 'n000025', 'n000026', 'n000027', 'n000028', 'n000030', 'n000031', 'n000032', 'n000033', 'n000034', 'n000035', 'n000036', 'n000037', 'n000038', 'n000039', 'n000041', 'n000042', 'n000043', 'n000044', 'n000045', 'n000046', 'n000047', 'n000048', 'n000049', 'n000050', 'n000051', 'n000052', 'n000053', 'n000054', 'n000055', 'n000056', 'n000057', 'n000058', 'n000059', 'n000060', 'n000061', 'n000062', 'n000063', 'n000064', 'n000065', 'n000066', 'n000067', 'n000068', 'n000069', 'n000070', 'n000071', 'n000072', 'n000073', 'n000074', 'n000075', 'n000076', 'n000077', 'n000079', 'n000080', 'n000081', 'n000083', 'n000084', 'n000085', 'n000086', 'n000087', 'n000088', 'n000089', 'n000090', 'n000091', 'n000092', 'n000093', 'n000094', 'n000095', 'n000096', 

In [ ]:
# 3. Load MobileNetV2 base model without top layers
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet',
                         pooling='avg')
base_model.trainable = False  # Freeze base model initially


In [ ]:
num_classes = len(class_names)
model = models.Sequential([
    base_model,
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

# 6. Initial training (feature extraction)
print("Starting initial training...")
model.fit(train_ds,
          epochs=EPOCHS_INITIAL)

Starting initial training...
Epoch 1/30
 13/164 ━━━━━━━━━━━━━━━━━━━━ 27:24 11s/step - accuracy: 0.1604 - loss: 4.5040

KeyboardInterrupt: 

In [ ]:
# 7. Fine-tuning: unfreeze last layers of base model
base_model.trainable = True

# Recompile with lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Starting fine-tuning...")
model.fit(train_ds,
          epochs=EPOCHS_FINE_TUNE)

# 8. Save the trained model
model.save("mobilenetv2_face_recognition.h5")
print("Model saved as mobilenetv2_face_recognition.h5")
